## Decision Trees in Practice


In this assignment we will explore various techniques for preventing overfitting in decision trees. We will extend the implementation of the binary decision trees that we implemented in the previous assignment. You will have to use your solutions from this previous assignment and extend them.

In this assignment you will:

Implement binary decision trees with different early stopping methods.

Compare models with different stopping parameters.

Visualize the concept of overfitting in decision trees.

Let's get started!

In [1]:
import pandas as pd 
import numpy as np 
pd.set_option('display.max_colwidth',-1)

## Load LendingClub Dataset

In [3]:
loans = pd.read_csv('lending-club-data.csv')

In [4]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x:+1 if  x==0 else -1)
del loans['bad_loans']

#3. We will be using the same 4 categorical features as in the previous assignment:



In [5]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

#Extract these feature columns and target column from the dataset, and discard the rest of the feature columns.
loans = loans[features + [target]]

In [6]:
loans.head(2)

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1


**Notes to people using other tools**

If you are using SFrame, proceed to the section "Subsample dataset to make sure classes are balanced".

If you are NOT using SFrame, download the list of indices for the training and validation sets:

module-6-assignment-train-idx.json.zip

module-6-assignment-validation-idx.json.zip

Then follow the following steps:

*Apply one-hot encoding to loans. Your tool may have a function for one-hot encoding.

*Load the JSON files into the lists train_idx and validation_idx.

*Perform train/validation split using train_idx and validation_idx. In Pandas, for instance:


In [7]:
# Apply one-hot encoding to loans. Your tool may have a function for one-hot encoding.


categorical_variables =[]
for feat_name,feat_type in zip(loans.columns,loans.dtypes):
    if feat_type==object: # In pandas dataframe string types shows as object 
        categorical_variables.append(feat_name)

#df['list_from_dict'] = [[x['name'] for x in list_dict] for list_dict in df['list_dicts']]

for feature in categorical_variables:
    loans_one_hot = loans[feature].apply(lambda x:{x:1})
    # the above o/p will give like :  1 {u' 60 months': 1}- so need to convert it like {' 60 months': 1}which is list of dicts
    loans_one_hot_encoded =loans_one_hot.values.tolist() # gives list of dict 
    loans_unpacked = pd.DataFrame(loans_one_hot_encoded) # gives a dataframe 
    
    # Change NaN's to 0's
    for columns in loans_unpacked.columns:
        loans_unpacked[columns]=loans_unpacked[columns].fillna(0)
        loans[columns] = loans_unpacked[columns].values
    del loans[feature]  # removing cols ['grade', 'sub_grade', 'home_ownership', 'purpose', 'term']
    


In [8]:
loans.head(2)

,safe_loans,A,B,C,D,E,F,G,36 months,60 months,...,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year,n/a
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Let's see what the feature columns look like now:

In [9]:
# Remove the response variable safe_loans'

features = loans.columns.drop('safe_loans')

In [10]:
print(len(features))
features

25


Index([u'A', u'B', u'C', u'D', u'E', u'F', u'G', u' 36 months', u' 60 months',
       u'MORTGAGE', u'OTHER', u'OWN', u'RENT', u'1 year', u'10+ years',
       u'2 years', u'3 years', u'4 years', u'5 years', u'6 years', u'7 years',
       u'8 years', u'9 years', u'< 1 year', u'n/a'],
      dtype='object')

In [11]:
##Load the JSON files into the lists train_idx and validation_idx.

# 1st read the indexes in a json file 
train_val=pd.read_json('module-6-assignment-train-idx.json')
validation_val=pd.read_json('module-6-assignment-validation-idx.json')

# list out the values which is ndarray
lst_train =train_val.values.tolist()
lst_validation = validation_val.values.tolist()
# flattening the list of list to single list 
train_idx = [item for sublist in lst_train for item in sublist]
validation_idx = [item  for sublist in lst_validation  for item in sublist]

In [12]:
#Perform train/validation split using train_idx and tvalidation_idx.

train_data = loans.iloc[train_idx]
validation_data = loans.iloc[validation_idx]
print len(train_data) , len(validation_data)

37224 9284


## Now proceed to the section "Early stopping methods for decision trees", skipping three sections below.

** the below sections are also skipped in assignment with graphlab**
#Subsample dataset to make sure classes are balanced
#Transform categorical data into binary features
#Train-validation split



# Early stopping methods for decision trees

In this section, we will extend the **binary tree implementation** from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a **maximum depth**. (set by parameter `max_depth`).
2. Reached a **minimum node size**. (set by parameter `min_node_size`).
3. Don't split if the **gain in error reduction** is too small. (set by parameter `min_error_reduction`).

For the rest of this assignment, we will refer to these three as **early stopping conditions 1, 2, and 3**.

## Early stopping condition 1: Maximum depth

Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this.  We will **alert you** when you reach a function that was part of the previous assignment so that you can simply copy and past your previous code.


## Early stopping condition 2: Minimum node size

The function **reached_minimum_node_size** takes 2 arguments:

1. The `data` (from a node)
2. The minimum number of data points that a node is allowed to split on, `min_node_size`.

This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the **decision_tree_create** function.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below.

In [13]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    ## YOUR CODE HERE
    if len(data) < min_node_size:
        True

**Quiz Question:** Given an intermediate node with 6 safe loans and 3 risky loans, if the min_node_size parameter is 10, what should the tree learning algorithm do next?



In [14]:
#options 
# Create a leaf and return it
# Continue building the tree by finding the best splitting feature
# a is correct 

## Early stopping condition 3: Minimum gain in error reduction

The function **error_reduction** takes 2 arguments:

1. The error **before** a split, `error_before_split`.
2. The error **after** a split, `error_after_split`.

This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the **decision_tree_create** function.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below. 

In [15]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    ## YOUR CODE HERE
    return error_before_split-error_after_split

**Quiz Question:** Assume an intermediate node has 6 safe loans and 3 risky loans. For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the minimum gain in error reduction parameter is set to 0.2, what should the tree learning algorithm do next?

In [16]:
# since all 4 features are exhausted and min gain not achieved . then we need to start other features from the root till
# we see the min gain in error reduction 
# check options given
#options 
#Create a leaf and return it
#Continue building the tree by using the splitting feature that gives 0.14 error reduction - correct

## Grabbing binary decision tree helper functions from past assignment

Recall from the previous assignment that we wrote a function `intermediate_node_num_mistakes` that calculates the number of **misclassified examples** when predicting the **majority class**. This is used to help determine which feature is best to split on at a given node of the tree.

**Please copy and paste your code for `intermediate_node_num_mistakes` here**.

In [17]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    ## YOUR CODE HERE
    safe =sum(labels_in_node==1)
    
    # Count the number of -1's (risky loans)
    ## YOUR CODE HERE
    risky = sum(labels_in_node==-1)
    
    # Return the number of mistakes that the majority classifier makes.
    ## YOUR CODE HERE
    if safe<risky:
        num_mistakes = safe
    else:
        num_mistakes = risky
    return num_mistakes

We then wrote a function best_splitting_feature that finds the best feature to split on given the data and a list of features to consider.

**Please copy and paste your best_splitting_feature code here.**

In [18]:
def best_splitting_feature(data ,features,target):
    best_feature = None # Keep track of the best feature
    best_error =10 # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.
    
    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data)) 
    
    #Step 1: Loop over each feature in the feature list
    for feat in features:
         #Step 2: Within the loop, split the data into two groups
        #The left split will have all data points where the feature value is 0
        left_split= data[data[feat]==0]
        # The right split will have all data points where the feature value is 1
        right_split = data[data[feat]==1]
        
        #Step 3: Calculate the number of misclassified examples in both groups of data and use the above formula 
        #to compute the classification error.
        left_misclassified = intermediate_node_num_mistakes(left_split[target])
        right_misclassified = intermediate_node_num_mistakes(right_split[target])
        error = (left_misclassified + right_misclassified) / num_data_points 
        
        #Step 4: If the computed error is smaller than the best error found so far, store this feature and its error.
        if (error < best_error):
            best_error = error
            best_feature= feat
    
    return best_feature # Return the best feature we found        

Finally, recall the function create_leaf from the previous assignment, which creates a leaf node given a set of target values.

**Please copy and paste your create_leaf code here.**



In [19]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True,
            'prediction':None}   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =    1      ## YOUR CODE HERE
    else:
        leaf['prediction'] =  -1        ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

## Incorporating new early stopping conditions in binary decision tree implementation


Now, you will implement a function that builds a decision tree handling the three early stopping conditions described in this assignment.  In particular, you will write code to detect early stopping conditions 2 and 3.  You implemented above the functions needed to detect these conditions.  The 1st early stopping condition, **max_depth**, was implemented in the previous assigment and you will not need to reimplement this.  In addition to these early stopping conditions, the typical stopping conditions of having no mistakes or no more features to split on (which we denote by "stopping conditions" 1 and 2) are also included as in the previous assignment.

**Implementing early stopping condition 2: minimum node size:**

* **Step 1:** Use the function **reached_minimum_node_size** that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the `min_node_size` argument.
* **Step 2:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.


**Implementing early stopping condition 3: minimum error reduction:**

**Note:** This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction.

* **Step 1:** Calculate the **classification error before splitting**.  Recall that classification error is defined as:

$$
\text{classification error} = \frac{\text{# mistakes}}{\text{# total examples}}
$$
* **Step 2:** Calculate the **classification error after splitting**. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
* **Step 3:** Use the function **error_reduction** to that you implemented earlier to write an if condition to detect whether  the reduction in error is less than the constant provided (`min_error_reduction`). Don't forget to use that argument.
* **Step 4:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.

Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [34]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same target value."                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if len(remaining_features) == 0:
        print "Stopping condition 2 reached. No remaining features."                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(data, min_node_size):          ## YOUR CODE HERE 
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)  ## YOUR CODE HERE
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split[target])   ## YOUR CODE HERE
    right_mistakes = intermediate_node_num_mistakes(right_split[target])  ## YOUR CODE HERE
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:       ## YOUR CODE HERE
        print "Early stopping condition 3 reached. Minimum error reduction."
        return create_leaf(target_values)  ## YOUR CODE HERE 
    
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split,remaining_features,target,
                                     current_depth+1 , max_depth,min_node_size,min_error_reduction)
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a function to count the nodes in your tree:

In [23]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [24]:
features_new = [col for col in train_data.drop('safe_loans',axis=1).columns]
features_new

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 ' 36 months',
 ' 60 months',
 'MORTGAGE',
 'OTHER',
 'OWN',
 'RENT',
 '1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year',
 'n/a']

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [35]:
small_decision_tree = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 2, 
                                        min_node_size = 10, min_error_reduction=0.0)
if count_nodes(small_decision_tree) == 7:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_decision_tree)
    print 'Number of nodes that should be there : 7' 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------------------------------

## Build a tree!

Now that your code is working, we will train a tree model on the **train_data** with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

**Warning**: This code block may take a minute to learn. 

In [36]:
my_decision_tree_new = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Split on feature < 1 year. (85, 11)
--------------------------------------------------------------------
Subtree, depth = 4 (85 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 4 (11 d

Let's now train a tree model ignoring early stopping conditions 2 and 3 so that we get the same tree as in the previous assignment. To ignore these conditions, we set min_node_size=0 and min_error_reduction=-1 (a negative value).

In [37]:
my_decision_tree_old = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. 

## Making predictions

Recall that in the previous assignment you implemented a function `classify` to classify a new point `x` using a given `tree`.

**Please copy and paste your `classify` code here**.

In [38]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            ### YOUR CODE HERE
            return classify(tree['right'], x, annotate)
               

Now, let's consider the first example of the validation set and see what the my_decision_tree_new model predicts for this data point.

In [39]:
my_valid = dict(zip(validation_data.columns, validation_data.iloc[0,:]))
my_valid


{' 36 months': 0.0,
 ' 60 months': 1.0,
 '1 year': 0.0,
 '10+ years': 0.0,
 '2 years': 1.0,
 '3 years': 0.0,
 '4 years': 0.0,
 '5 years': 0.0,
 '6 years': 0.0,
 '7 years': 0.0,
 '8 years': 0.0,
 '9 years': 0.0,
 '< 1 year': 0.0,
 'A': 0.0,
 'B': 0.0,
 'C': 0.0,
 'D': 1.0,
 'E': 0.0,
 'F': 0.0,
 'G': 0.0,
 'MORTGAGE': 0.0,
 'OTHER': 0.0,
 'OWN': 0.0,
 'RENT': 1.0,
 'n/a': 0.0,
 'safe_loans': -1.0}

In [40]:
type(validation_data.iloc[0,:])

pandas.core.series.Series

In [41]:
print 'Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0,:])

Predicted class: -1 


In [42]:
print 'Predicted class: %s ' % classify(my_decision_tree_new, validation_data.to_dict(orient='records')[0])

Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [43]:
classify(my_decision_tree_new, validation_data.iloc[0], annotate = True)

Split on  36 months = 0.0
Split on A = 0.0
At leaf, predicting -1


-1

Let's now recall the prediction path for the decision tree learned in the previous assignment, which we recreated here as my_decision_tree_old.


In [44]:
classify(my_decision_tree_old, validation_data.to_dict(orient='records')[0], annotate = True)

Split on  36 months = 0.0
Split on A = 0.0
Split on B = 0.0
Split on C = 0.0
Split on D = 1.0
Split on E = 0.0
At leaf, predicting -1


-1

Quiz Question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for validation_set[0] shorter, longer, or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

In [153]:
# As you can see from above Prediction path for my_decision_tree_old is longer than my_decision_tree_new for 
#validation_set[0]

Quiz Question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for any point always shorter, always longer, always the same, shorter or the same, or longer or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

In [152]:
# since we are ignoring min node size and error reduction for old decision tree thus path taken by my_decision_tree_old
# will always be longer it can also be verified from the above example where we took validation_set[10]

In [45]:
#for any point  - let's take few other points and test both the models 

In [48]:
classify(my_decision_tree_new, validation_data.to_dict(orient='records')[10], annotate = True)

Split on  36 months = 0.0
Split on A = 0.0
At leaf, predicting -1


-1

In [49]:
classify(my_decision_tree_old, validation_data.to_dict(orient='records')[10], annotate = True)

Split on  36 months = 0.0
Split on A = 0.0
Split on B = 0.0
Split on C = 0.0
Split on D = 0.0
Split on E = 1.0
At leaf, predicting -1


-1

Quiz Question: For a tree trained on any dataset using max_depth = 6, min_node_size = 100, min_error_reduction=0.0, what is the maximum number of splits encountered while making a single prediction?

In [50]:
#  6 which is depth of tree. that is the max split 

## Evaluating the model

Now let us evaluate the model that we have trained. You implemented this evaluation in the function `evaluate_classification_error` from the previous assignment.

**Please copy and paste your `evaluate_classification_error` code here**.

In [51]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction=[]
    for j in xrange(len(data)):
        #print i
        prediction.append(classify(tree, data.iloc[j]))

    #prediction = data.apply(lambda x: classify(tree, x))
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    num_errors = 0 
    for i in xrange(len(data)):
        if data[target].iloc[i]!=prediction[i]:
            num_errors+=1
    return num_errors/float(len(data))


Now, let's use this function to evaluate the classification error of my_decision_tree_new on the validation_set.

In [52]:
type(validation_data)

pandas.core.frame.DataFrame

In [53]:
evaluate_classification_error(my_decision_tree_new, validation_data, target)

0.3837785437311504

In [54]:
evaluate_classification_error(my_decision_tree_old, validation_data, target)

0.3837785437311504

Quiz Question: Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the **previous assignment**?

In [55]:
0.3837785437311504-0.3837785437311504  
# since the difference is 0 . hence classification accuracy is same for both 

0.0

# Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**).

Train three models with these parameters:

1. **model_1**: max_depth = 2 (too small)
2. **model_2**: max_depth = 6 (just right)
3. **model_3**: max_depth = 14 (may be too large)

For each of these three, we set `min_node_size = 0` and `min_error_reduction = -1`.

** Note:** Each tree can take up to a few minutes to train. In particular, `model_3` will probably take the longest to train.

In [56]:
model_1 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 2, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------------------------------

In [58]:
model_2 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. 

In [59]:
model_3 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 14, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Split on feature OTHER. (1692, 1)
--

## Evaluating the models
Let us evaluate the models on the train and validation data. Let us start by evaluating the classification error on the training data:

In [60]:
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data, target)
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data, target)
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data, target)

Training data, classification error (model 1): 0.400037610144
Training data, classification error (model 2): 0.381850419084
Training data, classification error (model 3): 0.374462712229


In [61]:
print "Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_data, target)
print "Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_data, target)
print "Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_data, target)

Validation data, classification error (model 1): 0.398104265403
Validation data, classification error (model 2): 0.383778543731
Validation data, classification error (model 3): 0.380008616975


**Quiz Question:** Which tree has the smallest error on the validation data?

**Quiz Question:** Does the tree with the smallest error in the training data also have the smallest error in the validation data?

**Quiz Question:** Is it always true that the tree with the lowest classification error on the **training** set will result in the lowest classification error in the **validation** set?


In [128]:
# Model 3 has smallest error on validation data

# Model 3 has smallest error in training set . Yes model 3 has smallest error in validation set also 

#NO, it may be due to overfitting 

### Measuring the complexity of the tree

Recall in the lecture that we talked about deeper trees being more complex. We will measure the complexity of the tree as

```
  complexity(T) = number of leaves in the tree T
```

Here, we provide a function `count_leaves` that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in `model_1`, `model_2`, and `model_3`. 

In [62]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [63]:
count_leaves(model_1)

4

In [64]:
count_leaves(model_2)

41

In [65]:
count_leaves(model_3)

341

**Quiz Question**: Which tree has the largest complexity?

**Quiz Question**: Is it always true that the most complex tree will result in the lowest classification error in the validation_set?



In [66]:
# complexity(T) = number of leaves in the tree T so higher leaves have higher complextiy i.e model 3 

# Not necessary ,higher complex trees have low train error but high validation n test error 

# Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**negative**, **just right**, and **too positive**).

Train three models with these parameters:
1. **model_4**: `min_error_reduction = -1` (ignoring this early stopping condition)
2. **model_5**: `min_error_reduction = 0` (just right)
3. **model_6**: `min_error_reduction = 5` (too positive)

For each of these three, we set `max_depth = 6`, and `min_node_size = 0`.

** Note:** Each tree can take up to 30 seconds to train.

In [68]:
model_4 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. 

In [71]:
model_5 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=0)

 --------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Split on feature < 1 year. (85, 11)
--------------------------------------------------------------------
Subtree, depth = 4 (85 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 4 (11 

In [69]:
model_6 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Early stopping condition 3 reached. Minimum error reduction.


### Calculate the accuracy of each model (model_4, model_5, or model_6) on the validation set.

In [72]:
print "Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data, target)
print "Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data, target)
print "Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data, target)

Validation data, classification error (model 4): 0.383778543731
Validation data, classification error (model 5): 0.383778543731
Validation data, classification error (model 6): 0.503446790177


Using the `count_leaves` function, compute the number of leaves in each of each models in (**model_4**, **model_5**, and **model_6**). 


In [73]:
print count_leaves(model_4) , count_leaves(model_5) , count_leaves(model_6)

41 13 1


**Quiz Question:** Using the complexity definition above, which model (**model_4**, **model_5**, or **model_6**) has the largest complexity?

Did this match your expectation?

**Quiz Question:** **model_4** and **model_5** have similar classification error on the validation set but **model_5** has lower complexity. Should you pick **model_5** over **model_4**?


In [141]:
# Largest complexity is model 4  

# yes it matched the expectation as it has ignored early stopping condition and allows tree to grow 

# doesn't matter which model you pick since error reduction is ignored in model 4 and its 0 in model 5 but since the
# no. of leaves in model 5 is less than model 4 so. we can pick model 5

# Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (**too small**, **just right**, and **just right**).

Train three models with these parameters:
1. **model_7**: min_node_size = 0 (too small)
2. **model_8**: min_node_size = 2000 (just right)
3. **model_9**: min_node_size = 50000 (too large)

For each of these three, we set `max_depth = 6`, and `min_error_reduction = -1`.

** Note:** Each tree can take up to 30 seconds to train.

In [74]:
model_7 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. 

In [75]:
model_8 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 2000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. 

In [76]:
model_9 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. 

Now, let us evaluate the models (**model_7**, **model_8**, or **model_9**) on the **validation_set**.

In [77]:
print "Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data, target)
print "Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data, target)
print "Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data, target)

Validation data, classification error (model 7): 0.383778543731
Validation data, classification error (model 8): 0.383778543731
Validation data, classification error (model 9): 0.383778543731


Using the `count_leaves` function, compute the number of leaves in each of each models (**model_7**, **model_8**, and **model_9**). 

In [78]:
print count_leaves(model_7) , count_leaves(model_8) , count_leaves(model_9)

41 41 41


**Quiz Question:** Using the results obtained in this section, which model (**model_7**, **model_8**, or **model_9**) would you choose to use?

In [79]:
# since all of them have same classifiction error and same no. of leaves so it is better to choose the model which 
#has just right quantity of min no. of nodes which is model 8